In [ ]:
import numpy as np
import json
import time
import scipy
import scipy.sparse
from trimesh import TriMesh
import glob
from numpy import *
np.set_printoptions(linewidth=2000, suppress=True)
from Extract_Transformation_matrix_minimize_SVD_singular_value_allPoses import *

In [ ]:
#### read obj file
base_dir="./datas/cube4/poses-1/"
filenames=glob.glob(base_dir+"*.obj")
print filenames
meshes_List=[]
for i in range(len(filenames)):
    meshes_List.append(TriMesh.FromOBJ_FileName(filenames[i]))

mesh0=meshes_List[0]
mesh1_list=meshes_List[1:]

groundtruth_names=glob.glob(base_dir+"cube-*.DMAT")
print groundtruth_names

gt_all=None
for i in range(len(groundtruth_names)):
    gt=load_DMAT(groundtruth_names[i])
    gt=gt.T
    gt=gt.reshape((-1,4,3))
    gt=np.transpose(gt, (0,2,1)).reshape((-1,12))
    if gt_all is None:
        gt_all=gt
    else:
        gt_all=np.hstack((gt_all, gt))
    

print gt_all.shape

ind=np.arange(10)


vertices1=vertices1_temp=np.hstack((np.asarray(mesh0.vs),np.ones((len(mesh0.vs),1))))
vertices2=vertices2_temp=np.asarray(mesh1_list[0].vs)
for i in range(1,len(mesh1_list)):
    vertices1=np.hstack((vertices1, vertices1_temp))
    vertices2=np.hstack((vertices2, np.asarray(mesh1_list[i].vs)))
print vertices1.shape
print vertices2.shape

M=12*len(mesh1_list)

In [ ]:
def null_space_matrix(A, eps=1e-12):
    #### modify from https://stackoverflow.com/questions/5889142/python-numpy-scipy-finding-the-null-space-of-a-matrix
    u, s, vh = scipy.linalg.svd(A)
# #     print u.shape
# #     print vh.shape
#     padding = (A.shape[1]-len(s))
#     null_mask = np.concatenate(((s <= eps), np.ones((padding,),dtype=bool)),axis=0)
#     null_space = scipy.compress(null_mask, vh, axis=0)
#     null_matrix= scipy.transpose(null_space)

    null_matrix=vh[-9:,:]
    return null_matrix.T

chosed_ind=1
mesh1=mesh1_list[chosed_ind]
vertices0=np.asarray(mesh0.vs)
vertices1=np.asarray(mesh1.vs)
print vertices0.shape
def func(q, V0, V1):
    obj=np.zeros(0)
    for i in range(len(V0)):
        v0=V0[i]
        v1=V1[i]
        v0_expand=np.zeros((3,12))
        for k in range(3):
            v0_expand[k, k*4:k*4+3]=v0
            v0_expand[k, k*4+3]=1
    
        Null_matrix=null_space_matrix(v0_expand)
        T=np.zeros(12).reshape((3,-1))
        T[:,:-1]=np.identity(3)
        T[:,-1]=v1-v0
        T=T.ravel()
        obj=np.concatenate((obj,(q-(T + Null_matrix.dot(np.dot(Null_matrix.T,q-T)))).ravel()))
        
    return obj
        
def compute_I_Projection_matrix(V0):
    sum_I_P=np.zeros((12,12))
    Identity=np.identity(12)
    for i in range(len(V0)):
        v0=V0[i]
        v0_expand=np.zeros((3,12))
        for k in range(3):
            v0_expand[k, k*4:k*4+3]=v0
            v0_expand[k, k*4+3]=1
        Null_matrix=null_space_matrix(v0_expand)
        
        P=Null_matrix.dot(Null_matrix.T)
        sum_I_P+=(Identity-P)
    s= scipy.linalg.svd(sum_I_P, compute_uv=False)
    return s
        
        
def solve(q0, V0, V1):
    res=scipy.optimize.least_squares(func, q0, args=(V0, V1),jac='3-point', method='trf')
    q=res["x"]
    cost=res["cost"]
    return q, cost


q_space=[]

for i in range(len(vertices1)):
    sum_I_P=np.zeros((12,12))
    count=0
    indices = mesh0.vertex_vertex_neighbors(i)
#     print indices
    indices=np.asarray(indices)
    if len(indices)>=3:
        v0=vertices0[i].reshape((1,3))
        v0_neighbor=vertices0[indices,:]
        v1=vertices1[i].reshape((1,3))
        v1_neighbor=vertices1[indices,:]
        
        V0=np.vstack((v0, v0_neighbor))
        V1=np.vstack((v1, v1_neighbor))
        
        s=compute_I_Projection_matrix(V0)
        if s[-1] < 1e-15:
            continue

        q0=np.random.random((12,))
        q, cost=solve(q0, V0, V1)
        q_space.append((q, cost))

        
    if i%100==0:
        print i
    
sort_q_space=sorted(q_space, key=lambda x: x[1])


In [ ]:
print [ err for t, err in sort_q_space[:100] ]

In [ ]:
def Convexhull_on_uncorrellated_space( data, threshold = 1e-6 ):
    X = data
    ## Subtract the average.
    Xavg = average( X, axis = 0 )[newaxis,:]
    Xp = X - Xavg
    U, s, V = linalg.svd( Xp, full_matrices = False, compute_uv = True )
    print s
    ## The first index less than threshold
    stop_s = len(s) - searchsorted( s[::-1], threshold )
    def restore( uncorrellated_data ):
        return ( dot( uncorrellated_data, V[:stop_s] ) + Xavg )
    
    print( "V[0]:", V[:2] )
    
    uncorrellated_data=dot(Xp , V[:stop_s].T)
    return uncorrellated_data

def uncorrellated_space( data, threshold = 1e-6 ):
    X = data
    ## Subtract the average.
    Xavg = average( X, axis = 0 )[newaxis,:]
    Xp = X - Xavg
    U, s, V = linalg.svd( Xp, full_matrices = False, compute_uv = True )
    print s
    ## The first index less than threshold
    stop_s = len(s) - searchsorted( s[::-1], threshold )
    def restore( uncorrellated_data ):
        return ( dot( uncorrellated_data, V[:stop_s] ) + Xavg )
    
    print( "V[0]:", V[:2] )
    
    def project( correllated_data ):
        return dot( correllated_data - Xavg, V[:stop_s].T)
    
    return project, restore
error=[]
data=[]
for i in range(len(sort_q_space)):
    data.append(sort_q_space[i][0])
    error.append(sort_q_space[i][1])
print np.asarray(data).shape
uncorrellated_data=Convexhull_on_uncorrellated_space(np.asarray(data)[np.asarray(error)<1e-5])
print uncorrellated_data.shape

In [ ]:
gt1=gt_all[:,chosed_ind*12:chosed_ind*12+12]
print gt1.shape

In [ ]:
gt_project, gt_restore = uncorrellated_space(gt1)
uncorralted_gt1=gt_project(gt1)
print uncorralted_gt1.shape
print np.allclose( gt_restore( gt_project(gt1) ), gt1 )

In [ ]:
np.array( [ v[1] for v in sort_q_space ] )[ np.argsort( ( ( data - gt_restore( gt_project( data ) ) )**2 ).sum(1) )[:99] ]

In [ ]:
recover_project, recover_unproject = uncorrellated_space( np.asarray(data)[ np.argsort( ( ( data - gt_restore( gt_project( data ) ) )**2 ).sum(1) )[:90] ] )

In [ ]:
# ########## visualize difference of recovered M and groundtruth M.
# import matplotlib.pyplot as plt
# import matplotlib.animation
# from mpl_toolkits.mplot3d import Axes3D

# fig= plt.figure()
# ax=fig.add_subplot(111, projection='3d')
# text = fig.suptitle('transformation matrix comparison')

# test_vals=np.array([
#         [0,0,0,1],
#         [1,0,0,1],
#         [0,1,0,1],
#         [0,0,1,1]
#     ])

# lb=-2
# ub=2

# recover_vals_list=[]
# gt_vals_list=[]

# res=res.reshape((-1, 12))
# gt_all=gt_all.reshape((-1, 12))


# frames=len(res)
# for num in range(frames):
#     resi=res[num]
#     gti=gt_all[num]
#     recover_vals=np.multiply(resi.reshape((1,3,4)), test_vals.reshape((-1,1,4))).sum(axis=-1)
#     gt_vals=np.multiply(gti.reshape((1,3,4)), test_vals.reshape((-1,1,4))).sum(axis=-1)
#     recover_vals_list.append(recover_vals)
#     gt_vals_list.append(gt_vals)


# def update_graph(num):
#     ax.clear()##### if you want to show accumulated data in one figure, comment this line.
    
#     recover_vals=recover_vals_list[num]
#     gt_vals=gt_vals_list[num]
    
    
    
#     for u1, v1, w1 in zip(test_vals[1:,0]-test_vals[0,0], test_vals[1:,1]-test_vals[0,1], test_vals[1:,2]-test_vals[0,2]):
#         graph=ax.quiver(test_vals[0,0], test_vals[0,1], test_vals[0,2], u1, v1, w1, pivot = 'tail', length=np.sqrt(u1**2+v1**2+w1**2), color='r')
    
    
#     for u1, v1, w1 in zip(recover_vals[1:,0]-recover_vals[0,0], recover_vals[1:,1]-recover_vals[0,1], recover_vals[1:,2]-recover_vals[0,2]):
#         graph=ax.quiver(recover_vals[0,0], recover_vals[0,1], recover_vals[0,2], u1, v1, w1, pivot = 'tail', length=np.sqrt(u1**2+v1**2+w1**2), color='g')
    
#     for u1, v1, w1 in zip(gt_vals[1:,0]-gt_vals[0,0], gt_vals[1:,1]-gt_vals[0,1], gt_vals[1:,2]-gt_vals[0,2]):
#         graph=ax.quiver(gt_vals[0,0], gt_vals[0,1], gt_vals[0,2], u1, v1, w1, pivot = 'tail', length=np.sqrt(u1**2+v1**2+w1**2), color='b')


#     ax.set_xlim(lb,ub)
#     ax.set_ylim(lb,ub)
#     ax.set_zlim(lb,ub)
#     ax.set_xlabel('X')
#     ax.set_ylabel('Y')
#     ax.set_zlabel('Z')
    
#     text.set_text('transformation matrix comparison={}'.format(num))
    
    
#     return text, graph


# ani = matplotlib.animation.FuncAnimation(fig, update_graph, frames, interval=50)

# plt.show()